# Rede Neural Convolucional

Redes Neurais Convolucionais (também chamadas de ConvNet) alavancam informações e são, portanto, muito adequadas para classificar imagens. Essas redes usam uma arquitetura ad hoc inspirada em dados biológicos retirados de dados fisiológicos esperimentos feitos no córtex visual. Como discutido, nossa visão é baseada em múltiplos níveis de córtex, cada um reconhecendo cada vez mais informação. Primeiro, vemos pixels únicos; então deles, nós reconhecemos simples formas geométricas. E então mais e mais elementos sofisticados, como obejtos, rostos, corpos humanos , animais e assim por diante.

Uma Rede Neural Convolucional Profunda (DCNN) consiste em muitas camadas de rede. Dois tipos diferentes de camadas, convolucional e pooling , são normalmente alternado. A profundidade de cada filtro aumenta da esquerda paraa a direita na rede. O ultimo estágio é tipicamente feito de uma ou mais camadas totalmente conectadas.

Existem três intuições chave além do ConvNet:
- Campos receptivos locais
- Pesos compartilhados 
- Pooling

# Campos Receptivos Locais

Se queremos preservar a informação espacial, então é conveniente representar cada imagem com uma matriz de pixels. Então, uma maneira simples de codificar a estrutura local é conectar uma submatriz de neurônios de entrada adjacentes em um único oculto
neurônio pertencente à próxima camada. Esse único neurônio oculto representa um campo receptivo local. Note que esta operação é chamada de convolução e dá o nome para esse tipo de rede.

# Pesos e Viés Compartilhados

Vamos supor que queremos nos afastar da representação de pixels em uma linha ganhando a capacidade de detectar o mesmo recurso independentemente do local onde é colocado na imagem de entrada. Uma intuição simples é usar o mesmo conjunto de pesos e bias para todos os neurônios nas camadas ocultas. Desta forma, cada camada aprenderá um conjunto de recursos latentes independentes da posição derivada da imagem

# Camada Pooling

Supondo que queremos resumir a saída de um mapa de recursos. Mais uma vez, nós podemos usar a contiguidade espacial da saída produzida a partir de um único mapa de recursos e agregar os valores de uma submatriz em um único valor de saída que descreve sinteticamente o significado associado a essa região física.

# Implementação com o Keras

Para definir o código LeNet(CNN para detectar o MNIST), usamos o módulo 2D convolucional, que é:

keras.layers.convolutional.Conv2D(filters,Kernel_size,padding='valid')

- Aqui , filters é o número de kernels de convolução a serem usados (por exemplo,dimensionalidade da saída), o kernel_size é um inteiro ou tupla/lista de dois inteiros, especificando a largura e altura da janela de convolução 2D (pode ser um único inteiro para especificar o mesmo valor para todas as dimensões espaciais), e padding = 'same' significa que o preenchimento é usado.

- Existem duas opções de padding: padding ='valid' significa que a convolução é computada apenas onde a entrada e o filtro se sobrepõem completamente e portanto a saída é menor que a entrada. Enquanto padding = 'same' significa que temos uma saída que é do memso tamanho a entrada , para a qual a área ao redor da entrada é preenchida com zeros.

Além disso,usamos um módulo MaxPooling2D: 

keras.layers.pooling.MaxPooling2D(pool_size=(2,2),strides=(2,2))

Aqui, pool_size = (2, 2) é uma tupla de dois inteiros representando os fatores por qual a imagem é reduzida verticalmente e horizontalmente. Então (2, 2) irá reduzir pela metade a imagem em cada dimensão e strides = (2, 2) é a passada usada para o processamento.

In [19]:
from keras import backend as k
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.datasets import mnist
from keras.utils import np_utils
from keras.optimizers import SGD,RMSprop,Adam
import numpy as np
import matplotlib.pyplot as plt

In [20]:
#Definindo a ConvNet

class LeNet:
    @staticmethod
    def build(input_shape,classes):
        model = Sequential()
        #CONV => RELU => POOL
        model.add(Conv2D(20,kernel_size=5,padding="same",
                                input_shape=input_shape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        
        #CONV => RELU => POOL
        model.add(Conv2D(50,kernel_size=5,border_mode="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        
        #Flatten => RELu layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        #softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

In [23]:
#REde e Training
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = Adam()
VALIDATION_SPLIT = 0.2
IMG_ROWS,IMG_COLS = 28,28 #input image dimensions
NB_CLASSES = 10 #numero de saídas
INPUT_SHAPE = (1,IMG_ROWS,IMG_COLS)

#dividindo os dados em treino e teste
(X_train,y_train),(X_test,y_test) = mnist.load_data()
k.set_image_dim_ordering("th")

#normalizando os dados
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

X_train = X_train[:, np.newaxis, :, :]
X_test = X_test[:, np.newaxis, :, :]
print(X_train.shape[0],'train samples')
print(X_train.shape[0],'test samples')

#converter vetores de  classe em matrizes de classe binária
y_train = np_utils.to_categorical(y_train,NB_CLASSES)
y_test = np_utils.to_categorical(y_test,NB_CLASSES)

#Inicializador do otimizador do modelo
model = LeNet.build(input_shape=INPUT_SHAPE,classes=NB_CLASSES)
model.compile(loss="categorical_crossentropy",optimizer=OPTIMIZER,
             metrics=["accuracy"])

network = model.fit(X_train,y_train,batch_size=BATCH_SIZE,
                   epochs=NB_EPOCH,verbose=VERBOSE,validation_split=VALIDATION_SPLIT)

score = model.evaluate(X_test,y_test,verbose=VERBOSE)
print("Test score:",score[0])
print("Test acc:",score[1])

60000 train samples
60000 test samples


/home/paic-jailson/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, kernel_size=5, padding="same")`
  


Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 88s 2ms/step - loss: 0.1831 - acc: 0.9429 - val_loss: 0.0598 - val_acc: 0.9812
Epoch 2/20
48000/48000 [==============================] - 87s 2ms/step - loss: 0.0484 - acc: 0.9843 - val_loss: 0.0491 - val_acc: 0.9842
Epoch 3/20
48000/48000 [==============================] - 87s 2ms/step - loss: 0.0342 - acc: 0.9892 - val_loss: 0.0325 - val_acc: 0.9904
Epoch 4/20
48000/48000 [==============================] - 88s 2ms/step - loss: 0.0234 - acc: 0.9928 - val_loss: 0.0355 - val_acc: 0.9896
Epoch 5/20
48000/48000 [==============================] - 87s 2ms/step - loss: 0.0170 - acc: 0.9943 - val_loss: 0.0353 - val_acc: 0.9894
Epoch 6/20
48000/48000 [==============================] - 87s 2ms/step - loss: 0.0132 - acc: 0.9954 - val_loss: 0.0419 - val_acc: 0.9867
Epoch 7/20
48000/48000 [==============================] - 88s 2ms/step - loss: 0.0109 - acc: 0.9961 - val_loss: 0.0330 - val_acc